In [1]:
# Z

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1686301395753_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package('pandas')
sc.install_pypi_package('matplotlib')
sc.install_pypi_package('fsspec')
sc.install_pypi_package('s3fs ')
sc.install_pypi_package('seaborn')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Merge the Hail dataset and Structure dataset

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load the data

In [4]:
s3_uri = 's3a://noaa-swdi-pds/hail-????.csv'

schema = StructType([
    StructField("ZTIME", StringType(), nullable=True),
    StructField("LON", DoubleType(), nullable=True),
    StructField("LAT", DoubleType(), nullable=True),
    StructField("WSR_ID", StringType(), nullable=True),
    StructField("CELL_ID", StringType(), nullable=True),
    StructField("RANGE", DoubleType(), nullable=True),
    StructField("AZIMUTH", DoubleType(), nullable=True),
    StructField("SEVPROP", DoubleType(), nullable=True),
    StructField("PROB", DoubleType(), nullable=True)
])

hail = (
    spark
    .read
    .csv(s3_uri, header=True, schema=schema)
    .where(F.col('LON').isNotNull())
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
s3_uri = 's3a://noaa-swdi-pds/structure-????.csv'

schema = StructType([
    StructField("ZTIME", StringType(), nullable=True),
    StructField("LON", DoubleType(), nullable=True),
    StructField("LAT", DoubleType(), nullable=True),
    StructField("WSR_ID", StringType(), nullable=True),
    StructField("CELL_ID", StringType(), nullable=True),
    StructField("RANGE", DoubleType(), nullable=True),
    StructField("AZIMUTH", DoubleType(), nullable=True),
    StructField("BASE_HEIGHT", DoubleType(), nullable=True),
    StructField("TOP_HEIGHT", DoubleType(), nullable=True),
    StructField("VIL", DoubleType(), nullable=True),
    StructField("MAX_REFLECT", DoubleType(), nullable=True),
    StructField("HEIGHT", DoubleType(), nullable=True)
])

structure = (
    spark
    .read
    .csv(s3_uri, header=True, schema=schema)
    .where(F.col('LON').isNotNull())
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Engineer datetime features

In [13]:
hail = (
    hail
    .withColumn('ZTIME', F.to_timestamp(F.col('ZTIME'), 'yyyyMMddHHmmss'))
    .withColumn('ZDAY', F.date_format(F.col('ZTIME'), 'yyyy-MM-dd'))
    .withColumn('ZYEAR', F.year('ZTIME'))
    .withColumn('ZMONTH', F.month('ZTIME'))
    .withColumn('ZDAY', F.dayofyear('ZTIME'))
    .withColumn('ZHOUR', F.hour('ZTIME'))
    .withColumnRenamed('RANGE', 'RANGE_HAIL')
    .withColumnRenamed('AZIMUTH', 'AZIMUTH_HAIL')
    .withColumnRenamed('LON', 'LON_HAIL')
    .withColumnRenamed('LAT', 'LAT_HAIL')
    .withColumnRenamed('SEVPROP', 'SEVPROB')
    .drop('ZTIME')
)

structure = (
    structure
    .withColumn('ZTIME', F.to_timestamp(F.col('ZTIME'), 'yyyyMMddHHmmss'))
    .withColumn('ZDAY', F.date_format(F.col('ZTIME'), 'yyyy-MM-dd'))
    .withColumn('ZYEAR', F.year('ZTIME'))
    .withColumn('ZMONTH', F.month('ZTIME'))
    .withColumn('ZDAY', F.dayofyear('ZTIME'))
    .withColumn('ZHOUR', F.hour('ZTIME'))
    .withColumnRenamed('RANGE', 'RANGE_STRUCTURE')
    .withColumnRenamed('AZIMUTH', 'AZIMUTH_STRUCTURE')
    .withColumnRenamed('LON', 'LON_STRUCTURE')
    .withColumnRenamed('LAT', 'LAT_STRUCTURE')
    .drop('ZTIME')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# hail.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# structure.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Merge the hail and structure data and filter `SEVPROB` < 0

In [9]:
# merged_df.filter(F.col('SEVPROB') == -999.0).limit(10).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
merge_cols = ['WSR_ID', 'CELL_ID', 'ZYEAR', 'ZMONTH', 'ZDAY', 'ZHOUR']

merged_df = (
    hail
    .join(structure, on=merge_cols, how='inner')
    .filter(
        (F.col('SEVPROB') >= 0) &
        (F.col('PROB') >= 0) &
        (F.col('RANGE_HAIL') >= 0) &
        (F.col('RANGE_STRUCTURE') >= 0) &
        (F.col('AZIMUTH_HAIL') >= 0) &
        (F.col('AZIMUTH_STRUCTURE') >= 0) &
        (F.col('BASE_HEIGHT') >= 0) &
        (F.col('TOP_HEIGHT') >= 0) &
        (F.col('VIL') >= 0) &
        (F.col('MAX_REFLECT') >= 0) &
        (F.col('HEIGHT') >= 0) &
        (F.col('TOP_HEIGHT') >= 0)
    )
#     .filter(F.col('ZYEAR') >= 2010)
    .withColumn('LON', (F.col('LON_HAIL') + F.col('LON_STRUCTURE'))/2)
    .withColumn('LAT', (F.col('LAT_HAIL') + F.col('LAT_STRUCTURE'))/2)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Parquet

In [ ]:
parquet_path = 's3://bdcc2023-jwong/merged-hail-structure/'
merged_df.write.parquet(parquet_path, partitionBy='ZYEAR')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
merged_dict = {}
merged_dict['row_count'] = merged_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
merged_dict['row_count']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…